In [1]:
'''
A Multilayer Perceptron implementation example using TensorFlow library.
This example is using the MNIST database of handwritten digits
(http://yann.lecun.com/exdb/mnist/)

Author: Aymeric Damien
Project: https://github.com/aymericdamien/TensorFlow-Examples/
'''

'\nA Multilayer Perceptron implementation example using TensorFlow library.\nThis example is using the MNIST database of handwritten digits\n(http://yann.lecun.com/exdb/mnist/)\n\nAuthor: Aymeric Damien\nProject: https://github.com/aymericdamien/TensorFlow-Examples/\n'

In [2]:
# Import MINST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

import tensorflow as tf


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
# Parameters
learning_rate = 0.001
training_iters = 200000
batch_size = 128
display_step = 10

# Network Parameters
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)
dropout = 0.75 # Dropout, probability to keep units

# tf Graph input
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32) #dropout (keep probability)

In [4]:
# Create some wrappers for simplicity
def conv2d(x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)


def maxpool2d(x, k=2):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                          padding='SAME')


# Create model
def conv_net(x, weights, biases, dropout):
    # Reshape input picture
    x = tf.reshape(x, shape=[-1, 28, 28, 1])

    # Convolution Layer
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    # Max Pooling (down-sampling)
    conv1 = maxpool2d(conv1, k=2)

    # Convolution Layer
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # Max Pooling (down-sampling)
    conv2 = maxpool2d(conv2, k=2)

    # Fully connected layer
    # Reshape conv2 output to fit fully connected layer input
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    # Apply Dropout
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output, class prediction
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

In [18]:
# Store layers weight & bias
weights = {
    # 5x5 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    # 5x5 conv, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    # fully connected, 7*7*64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    # 1024 inputs, 10 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([1024, n_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}
def multilabel(logits, labels):
    y = tf.nn.softmax(logits)
    cost = -tf.reduce_sum(labels * tf.log(y))
    return cost
# Construct model
# Construct model
pred = conv_net(x, weights, biases, dropout)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
#cost = tf.reduce_mean(multilabel(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

In [36]:
batch_x, batch_y = mnist.train.next_batch(1)
ou = conv_net(batch_x, weights, biases, dropout)
with tf.Session() as sess:
    sess.run(init)
    print(len(batch_y))
    print(len(batch_x))
    print(batch_y[0])
    #print(batch_x[0])
    print(len(batch_x[0]))
    print(sess.run(tf.argmax(batch_y, 1)))
    print(sess.run(tf.argmax(ou, 1)))
    print(sess.run(tf.equal(tf.argmax(ou, 1), tf.argmax(batch_y, 1))))
    print(sess.run(tf.reduce_mean(tf.cast(tf.equal(tf.argmax(ou, 1), tf.argmax(batch_y, 1)), tf.float32))))
    
    p = sess.run([optimizer], feed_dict={x: batch_x, y: batch_y,
                                       keep_prob: dropout})
    print(p)
    loss, acc = sess.run([cost, accuracy], feed_dict={x: batch_x,
                                                              y: batch_y,
                                                              keep_prob: 1.})
    print "Iter " + str(1*batch_size) + ", Minibatch Loss= " + \
          "{:.6f}".format(loss) + ", Training Accuracy= " + \
          "{:.5f}".format(acc)

1
1
[ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]
784
[2]
[2]
[ True]
1.0
[None]
Iter 128, Minibatch Loss= 0.000000, Training Accuracy= 1.00000


In [40]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    while step * 1 < 20000:
        batch_x, batch_y = mnist.train.next_batch(1)
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y,
                                       keep_prob: dropout})
        if step % 100 == 0:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([cost, accuracy], feed_dict={x: batch_x,
                                                              y: batch_y,
                                                              keep_prob: 1.})
            print "Iter " + str(step) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc)
        step += 1
    print "Optimization Finished!"

    # Calculate accuracy for 256 mnist test images
    print "Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: mnist.test.images[:256],
                                      y: mnist.test.labels[:256],
                                      keep_prob: 1.})

Iter 100, Minibatch Loss= 0.000000, Training Accuracy= 1.00000
Iter 200, Minibatch Loss= 21355.175781, Training Accuracy= 0.00000
Iter 300, Minibatch Loss= 0.000000, Training Accuracy= 1.00000
Iter 400, Minibatch Loss= 0.000000, Training Accuracy= 1.00000
Iter 500, Minibatch Loss= 0.000000, Training Accuracy= 1.00000
Iter 600, Minibatch Loss= 0.000000, Training Accuracy= 1.00000
Iter 700, Minibatch Loss= 34705.414062, Training Accuracy= 0.00000
Iter 800, Minibatch Loss= 41742.070312, Training Accuracy= 0.00000
Iter 900, Minibatch Loss= 0.000000, Training Accuracy= 1.00000
Iter 1000, Minibatch Loss= 0.000000, Training Accuracy= 1.00000
Iter 1100, Minibatch Loss= 6342.035156, Training Accuracy= 0.00000
Iter 1200, Minibatch Loss= 0.000000, Training Accuracy= 1.00000
Iter 1300, Minibatch Loss= 0.000000, Training Accuracy= 1.00000
Iter 1400, Minibatch Loss= 0.000000, Training Accuracy= 1.00000
Iter 1500, Minibatch Loss= 0.000000, Training Accuracy= 1.00000
Iter 1600, Minibatch Loss= 0.00000

KeyboardInterrupt: 